# Introduction to Tensorflow

## TensorFlow Graph

- Tensorflow 는 computational graph 를 미리 작성하여 전체 연산을 효율적으로 수행

- 자동으로 gradient 계산하여 model 을 optimize 

- multi-GPU 및 TPU 지원하여 병렬처리

## Tensorflow Graph 의 구성 요소

### Placeholder variable 
    - graph 에 input (X, y) 공급

### Model variables 
    - Optimize 대상 parameter 변수

### model 
    - placeholder variable  과 model variable 을 이용하여 output 을 계산하는 수학적 함수

### cost measure 방법 (loss function)
    - variable 을 optimize 하는 기준

### optimization method 
    - model 의 variable 을 update


In [1]:
import tensorflow as tf

### Placeholder variables

In [2]:
X= tf.placeholder(tf.float32, [None, 5])

y = tf.placeholder(tf.float32, [None, 2])

### Variables to be optimized

In [3]:
weights = tf.Variable(tf.zeros([5, 2]))

biases = tf.Variable(tf.ones([2]))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
x1 = tf.constant(5)
x2 = tf.constant(6)

result = tf.multiply(x1,x2)
print(result)

Tensor("Mul:0", shape=(), dtype=int32)


### tensorflow session

In [5]:
with tf.Session() as sess:
    output = sess.run(result)
    print(output)

30


### 변수 연산

In [6]:
y = tf.matmul(X, weights)  + 2 * biases
print(y)

Tensor("add:0", shape=(?, 2), dtype=float32)


In [7]:
init = tf.initialize_all_variables()                 # 변수 초기화 instance 선언

with tf.Session() as sess:
    sess.run(init)
    output = sess.run(y, feed_dict={X: [[1,2,3,4,5]]})
    print(output)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[2. 2.]]


## Example of Tensorflow Programming

In [8]:
# load mnist data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype("float32")
X_test  = X_test.reshape(-1, 784).astype("float32")

X_train /= 255.
X_test /= 255.

Y_train = tf.keras.utils.to_categorical(y_train)
Y_test = tf.keras.utils.to_categorical(y_test)

### 변수들 선언

In [9]:
# 변수들을 설정한다.
x = tf.placeholder(tf.float32, [None, 784])   # input data (X) 가 제공될 변수 설정

y_ = tf.placeholder(tf.float32, [None, 10])     # label data (y) 가 제공될 변수 설정

W = tf.Variable(tf.zeros([784, 10]))             # Dense (10) - output layer 의 neuron 수

b = tf.Variable(tf.zeros([10]))                      # output layer 의 bias

### model 수식 지정

In [10]:
y = tf.nn.softmax(tf.matmul(x, W) + b)        # Activation

### loss 함수와 사용할 optimizer 지정

In [11]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))  # loss function 선언 

train_step = tf.train.AdamOptimizer(0.5).minimize(cross_entropy)   # optimizer 설정

Instructions for updating:
Use tf.cast instead.


### session 생성

In [12]:
init = tf.initialize_all_variables()                 # 변수 초기화 instance 선언

sess = tf.Session()                                    # tensorflow session 설정
sess.run(init)                                            # 변수 초기화

### model train

In [16]:
# 경사하강법으로 모델을 학습시킨다

EPOCHS = 1
BATCH_SIZE = 512

for epoch in range(EPOCHS):                             # EPOCHS 수 만큼 iterrate
    
    batch_count = int(X_train.shape[0]/BATCH_SIZE)       # epoch 당 배치의 갯수

    for i in range(batch_count):
    
        batch_xs, batch_ys = X_train[i*BATCH_SIZE:i*BATCH_SIZE+BATCH_SIZE], \
                                        Y_train[i*BATCH_SIZE:i*BATCH_SIZE+BATCH_SIZE]           # mini-batch data 작성

        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})    # train 실행

        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        if i % 10 == 0 : 
            print(str(epoch+1)+' epoch '+ str(i)+' batch ', 
                          sess.run(accuracy, feed_dict={x: X_test[:BATCH_SIZE], y_: Y_test[:BATCH_SIZE]}))

1 epoch 0 batch  0.08203125
1 epoch 10 batch  0.08203125
1 epoch 20 batch  0.08203125
1 epoch 30 batch  0.08203125
1 epoch 40 batch  0.08203125
1 epoch 50 batch  0.08203125
1 epoch 60 batch  0.08203125
1 epoch 70 batch  0.08203125
1 epoch 80 batch  0.08203125
1 epoch 90 batch  0.08203125
1 epoch 100 batch  0.08203125
1 epoch 110 batch  0.08203125


In [17]:
# 학습된 모델이 얼마나 정확한지를 출력한다.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: X_test, y_: Y_test}))

0.098


# Introduction to keras

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation

# model 1 : No Hidden Layer
model = tf.keras.Sequential()
model.add(Dense(10, input_shape=(784,)))
model.add(Activation('softmax'))

In [19]:
EPOCHS = 1
BATCH_SIZE = 512

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=0, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test)

print("Test score :", score[0])          # evaluation of loss function
print("Test accuracy :", score[1])

10000/10000 [==============================] - 0s 50us/sample - loss: 0.6328 - acc: 0.8606
Test score : 0.632761088180542
Test accuracy : 0.8606
